In [22]:
from ase import atoms  # may also be worth checking for binneed to check for the binaries perhaps
from pyqstem import PyQSTEM
import pymatgen as pmg
import numpy as np
from pymatgen.io import ase as pymatgenase
from matplotlib import pyplot as plt
import hyperspy.api as hs
from pyqstem.imaging import CTF
from pyqstem.wave import view
% matplotlib tk

In [23]:
## Lifted from QSTEM_MWE

def convert_wave_to_diffraction_pattern_array(wave):
    array=wave.array
    img = (np.abs(np.fft.fftshift(np.fft.fft2(array))))**2
    #img = np.log(np.abs(np.fft.fftshift(np.fft.fft2(array))))**2
    extent=wave.get_reciprocal_extent()
    img=img[None,:,:]
    img=img[-1,:,:].T
    img[72][72] = 0 
    return img,extent 

In [24]:
Ga = pmg.Element("Ga")
As = pmg.Element("As")
lattice = pmg.Lattice.cubic(5.65)

structure = pmg.Structure.from_spacegroup("F-43m",lattice, [Ga,As], [[0, 0, 0],[0.5,0.5,0.5]])
structure.make_supercell([6,4,50]) #As in Table 7.2 and Figure 7.2 of Kirkland

In [34]:
accelerating_voltage = 200

qstem = PyQSTEM('TEM') #initialise a TEM object
qstem.set_atoms(pymatgenase.AseAtomsAdaptor.get_atoms(structure)) #this does a pymatgen ---> conversion
qstem.build_wave('plane',accelerating_voltage,(144,144)) ## where do these hardwired numbers come from
num_slices=100 ### what should this number be?
qstem.build_potential(num_slices)
qstem.run() ## this now means we have a wave that has passed through our sample
wave=qstem.get_wave()

exit,exit_extent = convert_wave_to_diffraction_pattern_array(wave)

In [35]:
ctf = CTF(defocus=500,Cs=5*10**4,focal_spread=30)
img_wave=wave.apply_ctf(ctf)
img,img_extent = convert_wave_to_diffraction_pattern_array(img_wave)

In [40]:
resample=.18 # Angstrom/pixel
dose=1*10**4 #electrons/Angstrom squared
dose = 0
img_down=img_wave.detect(resample=resample)

In [41]:
fig, ax = plt.subplots(figsize=(6,6))
imshow = ax.imshow(exit)
#plt.figure()

In [38]:
fig, ax = plt.subplots(figsize=(6,6))
imshow = ax.imshow(img)
#plt.figure()

In [42]:
fig, ax = plt.subplots(figsize=(6,6))
imshow = ax.imshow(img_down.T)

In [43]:
img_wave.detect?